In [34]:
import pandas as pd
import numpy as np
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
from scipy import stats
from sklearn.model_selection import train_test_split

In [47]:
con = psycopg2.connect(
    host = 'mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com',
    database = 'mid_term_project',
    user = 'lhl_student',
    password = 'lhl_student',
    port = 5432
)
sql_flight_info = "SELECT fl_date, mkt_unique_carrier, op_unique_carrier, op_carrier_fl_num, origin,crs_dep_time, dep_time, dep_delay, dest, arr_time, arr_delay, crs_arr_time, crs_elapsed_time, actual_elapsed_time, distance, carrier_delay, weather_delay, nas_delay, security_delay, late_aircraft_delay FROM flights ORDER BY RANDOM() LIMIT 1000000;"
data_flight_info = pd.read_sql_query(sql_flight_info,con)
con.close()

In [3]:
data_flight_info.to_csv('training_data.csv', encoding='utf-8', index=False)

In [48]:
# Creating a function which will remove extra leading 
# and tailing whitespace from the data.
# pass dataframe as a parameter here

def whitespace_remover(dataframe):
    
    # iterating over the columns
    for i in dataframe.columns:
          
        # checking datatype of each columns
        if dataframe[i].dtype == 'object':
              
            # applying strip function on column
            dataframe[i] = dataframe[i].map(str.strip)
        else:
              
            # if condn. is False then it will do nothing.
            pass

In [49]:
df_flight_info = data_flight_info.copy()
whitespace_remover(df_flight_info)
df_flight_info.shape

(365, 2)

In [50]:
# Creating year, month, date columns
# Create new columns
df_flight_info['year'] = pd.DatetimeIndex(df_flight_info['fl_date']).year
df_flight_info['month'] = pd.DatetimeIndex(df_flight_info['fl_date']).month
df_flight_info['day'] = pd.DatetimeIndex(df_flight_info['fl_date']).day
df_flight_info['weekday'] = ((pd.DatetimeIndex(df_flight_info['fl_date']).dayofweek) // 5 == 1).astype(float)
df_flight_info.isnull().sum()

KeyError: 'fl_date'

In [39]:
# Filling all NaN delay info with 0
df_flight_info[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']] = df_flight_info[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].fillna(0)

In [40]:
# Because dep_time, dep_delay, arr_time, arr_delay, and actual_elapsed_time have relatively small amounts of NaN they were dropped
df_flight_info = df_flight_info.dropna()

In [41]:
df_flight_info.isnull().sum()

fl_date                0
mkt_unique_carrier     0
op_unique_carrier      0
op_carrier_fl_num      0
origin                 0
crs_dep_time           0
dep_time               0
dep_delay              0
dest                   0
arr_time               0
arr_delay              0
crs_arr_time           0
crs_elapsed_time       0
actual_elapsed_time    0
distance               0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
year                   0
month                  0
day                    0
weekday                0
dtype: int64

In [42]:
# 20,494 rows removed
df_flight_info.shape

(980197, 24)

In [43]:
df_flight_info1 = df_flight_info[df_flight_info['carrier_delay'] <= 720]
df_flight_info2 = df_flight_info1[df_flight_info1['weather_delay'] <= 720]
df_flight_info3 = df_flight_info2[df_flight_info2['nas_delay'] <= 720]
df_flight_info4 = df_flight_info3[df_flight_info3['security_delay'] <= 720]
df_flight_info = df_flight_info4[df_flight_info4['late_aircraft_delay'] <= 720]
#630 rows removed

In [44]:
df_flight_info.shape

(979648, 24)

In [45]:
# #Only want top 19 busiest airports
# airports = ['ORD','ATL','DFW','DEN','LAX','CLT','SEA','SFO','EWR','LGA','PHX','IAH','LAS','BOS','MCO','DTW','PHIL','DCA','MSP']
# df_flight_info = df_flight_info[df_flight_info['origin'].isin(airports)]
# df_flight_info.origin.value_counts()

In [46]:
df_flight_info.to_csv('Regression_no_outliers_cleaned_data_03.csv, encoding='utf-8', index=False)